In [2]:
import json
import nltk
import spacy 
from datetime import datetime
import tweepy
import re
import string
import unicodedata
from gensim import corpora
from nltk.tokenize.toktok import ToktokTokenizer

In [3]:
# Let's use the Twitter Stream API to get tweets in real time
# We save our tweets to a file called "cats.json"
# override tweepy.StreamListener to add logic to on_status and on_error 
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status._json)
        with open ("insomnia.json", "a+") as stream_f:
            json_text = json.dumps(status._json)
            stream_f.write(json_text)
            stream_f.write('\n')
            
    def on_error(self, status_code):
        print("Error detected!")
        print (status_code)
        return False

In [4]:
def clean_tweet(tweet): 
    processed_tweet = {}
    processed_tweet["id"] = tweet['id']
    processed_tweet["user"] = tweet['user']['screen_name']
    processed_tweet["created_at"] = datetime.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y')
    created_at = datetime.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y')
    processed_tweet["lang"] = tweet['lang']
    
    if tweet['lang'] != "en":
        processed_tweet["is_en"] = False
    else: 
        processed_tweet["is_en"] = True
        
    if "full_text" in tweet: 
        processed_tweet["text"] = tweet['full_text']
    elif "extended_tweet" in tweet:
        processed_tweet["text"] = tweet["extended_tweet"]["full_text"]
    elif "text" in tweet: 
        processed_tweet["text"] = tweet['text']
        
    if 'retweeted_status' in tweet:
        rt = tweet['retweeted_status']
        processed_tweet["is_rt"] = True
        processed_tweet["rt_user"] = rt['user']['screen_name']
        processed_tweet["rt_id"] = rt['id']        
        if "full_text" in rt:
            processed_tweet["rt_text"] = rt['full_text']
        elif "extended_tweet" in tweet['retweeted_status']:
            processed_tweet["rt_text"] = tweet['retweeted_status']['extended_tweet']["full_text"]
        elif "text" in rt: 
            processed_tweet["rt_text"] = rt['text']
    else: 
        processed_tweet["is_rt"] = False
            
    return processed_tweet

In [5]:
info = {"consumer_key": "C85NCy1PkZiRnAwnS0Abt5FlF",
        "consumer_secret": "JS17fB3b4s4BmYZ74n4EgwRjG1w7kvChU2U1f1sC2zpXMxIDa9",
        "access_token": "41218838-OVyxdHIDYAyf1EBezFy9XZaT20qFyjfPw5H6MlnFM",
        "access_secret": "gDDDmbnMPeBI3SMrRheifA8QfmOZs7f24ab718C0BtsW2"}

In [6]:
auth = tweepy.OAuthHandler(info['consumer_key'], info['consumer_secret'])
auth.set_access_token(info['access_token'], info['access_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [7]:
#Collecting Twitter Data
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=['insomnia'])

{'created_at': 'Sat Apr 04 01:46:13 +0000 2020', 'id': 1246252702553837570, 'id_str': '1246252702553837570', 'text': 'my dad out here just woke up and is making coffee at 3.45am\n\nreminder; my dad is also an insomniac and my mom sleep… https://t.co/A7cHjVPk42', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1203359167848820737, 'id_str': '1203359167848820737', 'name': 'comrade himbo ☭', 'screen_name': 'gremlin_teeth', 'location': 'they/them ', 'url': 'http://linktr.ee/certifiedhimbo', 'description': '19 | trans | brown | kurd |\n\npls buy my art', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 43, 'friends_count': 37, 'listed_count': 0, 'favourites_count': 5841, 'statuses_count': 2649, 'created_at': 'Sat D

{'created_at': 'Sat Apr 04 01:46:16 +0000 2020', 'id': 1246252715836977153, 'id_str': '1246252715836977153', 'text': 'RT @InayahLamis: Insomnia. https://t.co/0BN6ip2zsQ', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 990627093598781445, 'id_str': '990627093598781445', 'name': 'Ifeoma', 'screen_name': 'NattithaBaddi', 'location': '✈️', 'url': 'http://www.instagram.com/itsiogbatue', 'description': 'Challenge Accepted.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 124, 'friends_count': 169, 'listed_count': 0, 'favourites_count': 17702, 'statuses_count': 2283, 'created_at': 'Sun Apr 29 16:21:18 +0000 2018', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': Fals

{'created_at': 'Sat Apr 04 01:46:23 +0000 2020', 'id': 1246252743741845509, 'id_str': '1246252743741845509', 'text': 'RT @InayahLamis: Insomnia. https://t.co/0BN6ip2zsQ', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1131252794684530690, 'id_str': '1131252794684530690', 'name': 'Ray Capone ♠️❤️', 'screen_name': 'RayCapone7', 'location': None, 'url': None, 'description': 'ACEEWORLD🕊♠️2ENI0R 👩🏽\u200d🎓🥳', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 196, 'friends_count': 482, 'listed_count': 0, 'favourites_count': 1674, 'statuses_count': 3328, 'created_at': 'Wed May 22 17:37:38 +0000 2019', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translato

{'created_at': 'Sat Apr 04 01:46:25 +0000 2020', 'id': 1246252752923222018, 'id_str': '1246252752923222018', 'text': 'RT @InayahLamis: Insomnia. https://t.co/0BN6ip2zsQ', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1125233849418559488, 'id_str': '1125233849418559488', 'name': 'Jenn Moya', 'screen_name': 'jenn140318', 'location': 'Pachuca de Soto, Hidalgo', 'url': None, 'description': '“the universe won’t notice bitch, you’re just another life, the sun will rise, the stars have already die...” 💀', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 6, 'friends_count': 111, 'listed_count': 0, 'favourites_count': 679, 'statuses_count': 170, 'created_at': 'Mon May 06 03:00:30 +0000 2019', 'utc_offset': None, 'time

{'created_at': 'Sat Apr 04 01:46:34 +0000 2020', 'id': 1246252790038573062, 'id_str': '1246252790038573062', 'text': 'RT @InayahLamis: Insomnia. https://t.co/0BN6ip2zsQ', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 959259467853508608, 'id_str': '959259467853508608', 'name': '1/17♑️', 'screen_name': 'xxgreggg', 'location': 'Dmv', 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 44, 'friends_count': 79, 'listed_count': 0, 'favourites_count': 2487, 'statuses_count': 1586, 'created_at': 'Fri Feb 02 02:57:34 +0000 2018', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F

...

{'created_at': 'Sat Apr 04 01:52:33 +0000 2020', 'id': 1246254295948943361, 'id_str': '1246254295948943361', 'text': 'RT @TruemanBradley: @Madonna “Those who know can’t sleep.” Soon the entire world will have insomnia. #ThesePeopleAreSick https://t.co/AEdCL…', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 69441103, 'id_str': '69441103', 'name': 'Ombudsman', 'screen_name': 'DrWallenberg', 'location': 'Antarctica', 'url': None, 'description': 'Tres bon', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 155, 'friends_count': 575, 'listed_count': 0, 'favourites_count': 23164, 'statuses_count': 11923, 'created_at': 'Thu Aug 27 23:36:13 +0000 2009', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang

{'created_at': 'Sat Apr 04 01:52:39 +0000 2020', 'id': 1246254323471966209, 'id_str': '1246254323471966209', 'text': '🔥', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1566116029, 'id_str': '1566116029', 'name': 'kilo', 'screen_name': 'GodSon_96', 'location': '💰🏃🏽', 'url': None, 'description': '|sc : godsgift_96|', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1804, 'friends_count': 1178, 'listed_count': 1, 'favourites_count': 4275, 'statuses_count': 11226, 'created_at': 'Wed Jul 03 15:53:51 +0000 2013', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http

{'created_at': 'Sat Apr 04 01:52:44 +0000 2020', 'id': 1246254344535568384, 'id_str': '1246254344535568384', 'text': 'RT @InayahLamis: Insomnia. https://t.co/0BN6ip2zsQ', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1035461628, 'id_str': '1035461628', 'name': 'kj.', 'screen_name': 'kjtsekona', 'location': None, 'url': None, 'description': '🇹🇴', 'translator_type': 'regular', 'protected': False, 'verified': False, 'followers_count': 1068, 'friends_count': 339, 'listed_count': 9, 'favourites_count': 61611, 'statuses_count': 48030, 'created_at': 'Tue Dec 25 19:39:07 +0000 2012', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': '1A1B1F', 'pro

{'created_at': 'Sat Apr 04 01:52:56 +0000 2020', 'id': 1246254393609138176, 'id_str': '1246254393609138176', 'text': 'You have to love her. No other option.', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1220537680653582336, 'id_str': '1220537680653582336', 'name': 'Maybe: Ty🦋🖤', 'screen_name': 'TySenpai_', 'location': 'Limon, Costa Rica', 'url': 'https://www.instagram.com/xo._charisse_/', 'description': 'I said what I said bitch. 🖕🏽 Sag ☀️ Virgo 🌙 Cap ↗️', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 98, 'friends_count': 77, 'listed_count': 0, 'favourites_count': 1804, 'statuses_count': 895, 'created_at': 'Fri Jan 24 02:44:02 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lan

KeyboardInterrupt: 

In [13]:
# load the raw tweets found from the search and stream apis into the same list for processing
tweet_data = []

with open("insomnia.json") as stream_f:
    for line in stream_f:
        json_line = json.loads(line)
        tweet_data.append(json_line)
        
# filter all of the raw tweets by turning them into clean_tweet objects
# the filtering is taken care of in the class function
filtered_data = []
for elem in tweet_data: 
    filtered_tweet = clean_tweet(elem)
    filtered_data.append(filtered_tweet)

In [14]:
filtered_data

[{'id': 1246239241534951424,
  'user': 'fixiontheboy',
  'created_at': datetime.datetime(2020, 4, 4, 0, 52, 43),
  'lang': 'en',
  'is_en': True,
  'text': 'Well shit den. Insomnia is a bellend. How do you guys do this',
  'is_rt': False},
 {'id': 1246239251286630401,
  'user': '420ogstunner',
  'created_at': datetime.datetime(2020, 4, 4, 0, 52, 46),
  'lang': 'en',
  'is_en': True,
  'text': 'RT @Medicat67671113: Blue Cheese is an indica Cross of Blueberry/Cheese strains from UK. The heavy indica effects will help you feel relaxe…',
  'is_rt': True,
  'rt_user': 'Medicat67671113',
  'rt_id': 1226156420552429576,
  'rt_text': 'Blue Cheese is an indica Cross of Blueberry/Cheese strains from UK. The heavy indica effects will help you feel relaxed at the end of the day//can provide relief for insomnia,stress, depression, bipolar, PTSD,It has a Sweet,blueberry flavor &amp; THC is  22%+. https://t.co/UJ6YV0T8oR https://t.co/uRW3y9ko8O'},
 {'id': 1246239263848611841,
  'user': 'avibecalledti

In [15]:
# create a list of all the tweet text 
# we filter out all tweets that are not English
tweet_text = []
for tweet in filtered_data:
    if tweet["is_en"]:
        if tweet["is_rt"]: 
            tweet_text.append(tweet["rt_text"].replace("\n", " "))
        else:
            tweet_text.append(tweet["text"].replace("\n", " "))

In [16]:
tweet_text

['Well shit den. Insomnia is a bellend. How do you guys do this',
 'Blue Cheese is an indica Cross of Blueberry/Cheese strains from UK. The heavy indica effects will help you feel relaxed at the end of the day//can provide relief for insomnia,stress, depression, bipolar, PTSD,It has a Sweet,blueberry flavor &amp; THC is  22%+. https://t.co/UJ6YV0T8oR https://t.co/uRW3y9ko8O',
 '@temsss__ insomnia?',
 'Insomnia. https://t.co/0BN6ip2zsQ',
 'Banana OG is an indica-dominant cross of OG Kush and Banana,THC level of 19%, making it ideal for long-time tokers and those new or sensitive to cannabis. It can treat chronic pain, depression,insomnia, and chronic fatigue. best for evening usage. https://t.co/UJ6YV0T8oR https://t.co/CQ4ZYRL3nF',
 'i didnt know what to do with my insomnia and anxiety jud last night😆 yawa kamatay',
 'Insomnia. https://t.co/0BN6ip2zsQ',
 'Insomnia. https://t.co/0BN6ip2zsQ',
 '@Madonna “Those who know can’t sleep.” Soon the entire world will have insomnia. #ThesePeopleAr

In [17]:
#PreProcessing Data
# remove HTML links, mentions, hashtags, and special characters

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ' ')    
    return text

def strip_mentions(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def strip_hashtags(text):
    entity_prefixes = ['#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)
        
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [18]:
stripped_tweet_text = []
for elem in tweet_text:
    elem = strip_links(elem)
    elem = strip_mentions(elem)
    elem = strip_hashtags(elem)
    elem = elem.replace('RT', '')
    elem = remove_special_characters(elem)
    stripped_tweet_text.append(elem)

In [19]:
stripped_tweet_text

['Well shit den Insomnia is a bellend How do you guys do this',
 'Blue Cheese is an indica Cross of Blueberry Cheese strains from UK The heavy indica effects will help you feel relaxed at the end of the day can provide relief for insomnia stress depression bipolar PTSD It has a Sweet blueberry flavor amp THC is 22',
 'insomnia',
 'Insomnia',
 'Banana OG is an indica dominant cross of OG Kush and Banana THC level of 19 making it ideal for long time tokers and those new or sensitive to cannabis It can treat chronic pain depression insomnia and chronic fatigue best for evening usage',
 'i didnt know what to do with my insomnia and anxiety jud last night yawa kamatay',
 'Insomnia',
 'Insomnia',
 'Those who know cant sleep  Soon the entire world will have insomnia ThesePeopleAreSick',
 'Those who know cant sleep  Soon the entire world will have insomnia ThesePeopleAreSick',
 'Insomnia',
 'Insomnia',
 'Insomnia',
 '2am thank u lord for the insomnia I am receiving',
 'Shiro I will name this s

In [20]:
from textblob import TextBlob

for elem in stripped_tweet_text:
    print (elem)
    elem_textblob = TextBlob(elem)
    print (elem_textblob.sentiment)
    print ("----")

Well shit den Insomnia is a bellend How do you guys do this
Sentiment(polarity=-0.2, subjectivity=0.8)
----
Blue Cheese is an indica Cross of Blueberry Cheese strains from UK The heavy indica effects will help you feel relaxed at the end of the day can provide relief for insomnia stress depression bipolar PTSD It has a Sweet blueberry flavor amp THC is 22
Sentiment(polarity=0.03749999999999999, subjectivity=0.3125)
----
insomnia
Sentiment(polarity=0.0, subjectivity=0.0)
----
Insomnia
Sentiment(polarity=0.0, subjectivity=0.0)
----
Banana OG is an indica dominant cross of OG Kush and Banana THC level of 19 making it ideal for long time tokers and those new or sensitive to cannabis It can treat chronic pain depression insomnia and chronic fatigue best for evening usage
Sentiment(polarity=0.3477272727272727, subjectivity=0.509090909090909)
----
i didnt know what to do with my insomnia and anxiety jud last night yawa kamatay
Sentiment(polarity=0.0, subjectivity=0.06666666666666667)
----
Ins

In [22]:
#find sentiment vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [23]:
snt = analyser.polarity_scores('This is an examle of a happy tweet')
print(snt)

{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'compound': 0.5719}


In [24]:
snt = analyser.polarity_scores('This is an examle of a happy tweet')
print(snt)

{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'compound': 0.5719}
